In [ ]:
from gensim.models import KeyedVectors
wv = KeyedVectors.load_word2vec_format(r'D:\数据集\谷歌新闻word2dev\GoogleNews-vectors-negative300.bin.gz', binary=True)

In [ ]:
from nlpia.data.loaders import get_data
cities = get_data('cities')
cities.head(1).T

In [ ]:
import pandas as pd
us = cities[(cities.country_code == 'US') & (cities.admin1_code.notnull())].copy()
states = pd.read_csv(r'D:\数据集\美国城市\states.csv')
states = dict(zip(states.Abbreviation, states.State))
us['city'] = us.name.copy()
us['st'] = us.admin1_code.copy()
us['state'] = us.st.map(states)
us[us.columns[-3:]].head()

In [ ]:
import numpy as np
vocab = pd.np.concatenate([us.city, us.st, us.state])


In [ ]:
vocab = np.array([word for word in vocab if word in wv])
vocab[:5]

In [ ]:
city_plus_state = []
us = us.sort_values('population', ascending=False)
for c, state, st in zip(us.city, us.state, us.st):
    if c not in vocab:
        continue
    row = []
    if state in vocab:
        row.extend(wv[c] + wv[state])
    else:
        row.extend(wv[c] + wv[st])
    city_plus_state.append(row)
us_300D_sorted = pd.DataFrame(city_plus_state)
del city_plus_state
del wv


In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2) 
us_300D = get_data('cities_us_wordvectors')
us_2D = pca.fit_transform(us_300D.iloc[:10, :300])

In [ ]:
#这段代码已然失效,如果要看图还是需要自己画一个.
from nlpia.data.loaders import get_data
from nlpia.plots import offline_plotly_scatter_bubble
df = get_data('cities_us_wordvectors_pca2_meta')
df = df.sort_values('population', ascending=False)[:10].copy()
df[['x', 'y']] = - df[['x', 'y']]  # <1>
html = offline_plotly_scatter_bubble(
    df, x='x', y='y',
    size_col=None, text_col='name', category_col=None,
    xscale=None, yscale=None,  # 'log' or None
    layout={}, marker={'sizeref': 3000})
with open(r'D:\中间结果\wordmap.html', 'w') as fout:
    fout.write(html)